# NMR Inversion Recovery Analysis in Python

* **Author:** Seth Veenbaas
* **Last Updated:** 12/18/24

## Objective
In this notebook, we will learn the basics of working with scientific data in Python, focusing on an NMR inversion recovery experiment. We will import experimental data into a Pandas DataFrame, calculate the T<sub>1</sub> relaxation times and ideal an ideal d<sub>1</sub> delay time, and visualize the fitted data.

## Inversion recovery experiment
The inversion-recovery experiment measures T1 relaxation times of any nucleus. If the net magnetization is placed along the -z axis, it will gradually return to its equilibrium position along the +z axis at a rate governed by T1. The equation governing this behavior as a function of the time t after its displacement is: 

M<sub>z</sub>(t) = M<sub>z, eq</sub> * (1 - 2 e<sup>-t/T<sub>1</sub></sup>)

The basic pulse sequence consists of an 180<sup>°</sup> pulse that inverts the magnetization to the -z axis. During the following delay, relaxation along the longitudinal plane takes place. Magnetization comes back to the original equilibrium z-magnetization. A 90<sup>°</sup> pulse creates transverse magnetization. The experiment is repeated for a series of delay values taken from a variable delay list. A 1D spectrum is obtained for each value of vd and stored in a pseudo 2D dataset. The longer the relaxation delay (d<sub>1</sub>) is, the more precise the T<sub>1</sub> measurement is. An ideal relaxation time (d<sub>1</sub>) can be calculated (aq = acquisition time):

 d<sub>1</sub> + aq = 5 * T<sub>1</sub>

![image.png](images/t1_relaxation_pulse_sequence.png)

More information:
https://imserc.northwestern.edu/downloads/nmr-t1.pdf

---

## 1. Importing Required Libraries

First, let's import the python libraries/packages we need to work with the data.


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import os
import mnova

# Enable inline plotting
%matplotlib inline

# Set DataFrame precision to 3 decimal places
pd.set_option("display.precision", 3)


---
## 2. Importing the Data

We will now import the NMR inversion recovery data from a CSV file to a Pandas Dataframe (the excel of Python).

The file `Ibuprofen-C13-invrec-data-mnova.csv` contains the experimental data from an inversion recovery experiment for ibuprofen.

In [ ]:
# Load the data from the CSV file
df = pd.read_csv('data/Ibuprofen-C13-invrec-data-mnova.csv', header=1)

# Display the first 5 row of the dataframe to understand its structure
df.head(5)

### Use the `mnova.rename_columns()` function to reformat the dataframe into a more user friendly format.

|       Time       |            <#>_ppm           |
| :--------------: | :--------------------------: |
| Time of scan (s) | Chemical shift of each peak. |

```python
output = function_name(input)
```

In [ ]:
# Runs reformating function
df = mnova.rename_columns(df)

# Display the full dataframe
df

---

## 3. Visualizing the Data

We are going to use MatPlotLib to generate visualization for multiple all the peaks in our data. To make multiple plots efficiently we can create a reusable plotting function instead of type the same code multiple times.

### How to Define a Function
A function in Python is defined using the def keyword, followed by:

1) **Function name**: A descriptive identifier (e.g., calculate_energy).
2) **Parentheses**: Containing optional parameters (inputs).
3) **Colon**: Signals the beginning of the function's body.
4) **Indented code block**: Contains the instructions the function will execute.

Here is what it looks like to define a function:

```python
def function_name(parameters):
    # Function body (indented code)
    return output  # Optional: Return a result
```

Let's create a function that plots data from our NMR Dataframe!


In [ ]:
def plot_nmr_data(df : pd.DataFrame):
    plt.figure(figsize=(10, 8))
    
    for column in df.columns:
        if column != 'Time':
            plt.plot(df['Time'], df[column], label=column)
    
    plt.xlabel('Time (s)')
    plt.ylabel('Signal')
    plt.title('NMR Inversion Recovery Data')
    plt.legend()
    plt.show()

# Visualize the data
plot_nmr_data(df)


---

## 4. Calculating T1 Relaxation Time

To calculate the T1 relaxation time for each peak, we will fit the data to the inversion recovery model:

M<sub>z</sub>(t) = M<sub>z, eq</sub> * (1 - 2 e<sup>-t/T1</sup>) + C


In [ ]:
# Function to model the inversion recovery curve
def inversion_recovery(t, M, T1, C):
    return M * (1 - 2 * np.exp(-t / T1)) + C

# Function to calculate T1 relaxation time for a given nucleus
def calculate_T1(df : pd.DataFrame, x_col : str, y_col : str):
    t = df[x_col].values
    y = df[y_col].values
    
    # Initial guess for A, T1, and C
    initial_guess = [max(y), 1, min(y)]
    
    # Fit the curve
    popt, _ = curve_fit(inversion_recovery, t, y, p0=initial_guess)
    
    # Return the fitted parameters and T1 value
    return popt, popt[1]


---

## 5. Visualizing the Fitted Data

Let's create a function to plot the data and the fitted curve.


In [ ]:
def plot_fitted_data(df : pd.DataFrame, x_col : str, y_col : str, popt : np.ndarray):
    x = df[x_col].values
    y = df[y_col].values
    
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, label=f'{y_col} Data')
    plt.plot(x, inversion_recovery(x, *popt), label=f'Fit: T1 = {popt[1]:.3f} s', color='red')
    plt.xlabel('Time (s)')
    plt.ylabel('Signal')
    plt.title(f'Inversion Recovery Fit for {y_col}')
    plt.legend()
    plt.show()


---

## 6. Analyzing and Summarizing T1 Times for All Nuclei

Now, let's use the functions we created to calculate the T1 relaxation time for each nucleus and visualize the fits.


In [ ]:
# Dictionary to store the T1 times
T1_times = {}

# Calculate T1 for each nucleus and plot the fit
for column in df.columns:
    if column != 'Time':
        popt, T1 = calculate_T1(df, 'Time', column)
        T1_times[column] = T1
        plot_fitted_data(df, 'Time', column, popt)

# Create a summary table of T1 times
T1_df = pd.DataFrame(list(T1_times.items()), columns=['Peak', 'T1_Time'])
T1_df


---

## 7. Calculate an ideal delay time (d<sub>1</sub>)

Now, let's calculate an ideal delay time (d<sub>1</sub>) for future NMR experiments on ibuprofen based on the (T<sub>1</sub>) times we measured.

d<sub>1</sub> + aq = 5 * T<sub>1</sub>

aq (acquisition time) = 1.3 seconds (Bruker default for C<sub>13</sub>)

Relaxation time (d<sub>1</sub>) can not be shorter than 0 seconds:

* if d<sub>1</sub> > 0 seconds:

    d<sub>1</sub> = (5 * T<sub>1</sub>) - aq

* else:

    d<sub>1</sub> = 0 seconds

In [ ]:
# Define the conditional logic for calculating the relaxation time (d1)
def calculate_d1_time(t1_time : float, aq : float):
    d1 = 5 * t1_time - aq
    if d1 > 0:
        return d1
    else:
        return 0

T1_df['D1_Time'] = T1_df['T1_Time'].apply(calculate_d1_time, aq=1.3)

# Find the largest D1
max_d1 = T1_df['D1_Time'].max()

# Round d1 time to nearest second
max_d1 = round(max_d1)

# Print the ideal d1 time using an f-string
print(f'Ideal d1 time: {max_d1} (s)')
T1_df

---

## 8. Export results

We can save the T<sub>1</sub> and d<sub>1</sub> values we calculated by exporting our summary DataFrame as a .csv file.

You can open the .csv file in excel.

In [ ]:
# Define output directory
directory = 'results'

# Make sure the parent directory exists
if not os.path.isdir(directory):
    os.makedirs(directory)

# Export DataFrame to a .csv file
T1_df.to_csv(f'{directory}/ibprofen_t1_d1_summary.csv')